In [ ]:
!pip install -q transformers datasets evaluate sacrebleu accelerate
!apt install git-lfs

In [ ]:
!pip install -U "protobuf==3.20.3"

In [ ]:
import os
from datasets import Dataset, DatasetDict

base_path = "/kaggle/input/vlsp-dataset"
train_en_path = os.path.join(base_path, "train.en.txt")
train_vi_path = os.path.join(base_path, "train.vi.txt")
test_en_path = os.path.join(base_path, "public_test.en.txt")
test_vi_path = os.path.join(base_path, "public_test.vi.txt")

In [ ]:
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines()
    return lines

In [ ]:
print("Đang đọc dữ liệu...")
train_en = read_text_file(train_en_path)
train_vi = read_text_file(train_vi_path)
val_en = read_text_file(test_en_path)
val_vi = read_text_file(test_vi_path)

print(f"Số câu Train En: {len(train_en)}")
print(f"Số câu Train Vi: {len(train_vi)}")
assert len(train_en) == len(train_vi), "Lỗi: Số dòng file Anh và Việt không khớp nhau!"

In [ ]:
train_dataset = Dataset.from_dict({"en": train_en, "vi": train_vi})
val_dataset = Dataset.from_dict({"en": val_en, "vi": val_vi})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
})

print("Load dữ liệu thành công:")
print(raw_datasets)

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    inputs = examples["en"]
    targets = examples["vi"]
    
    tokenizer.src_lang = "eng_Latn"
    tokenizer.tgt_lang = "vie_Latn"

    model_inputs = tokenizer(
        inputs,                      
        text_target=targets,          
        max_length=128,
        truncation=True
    )
    
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
import evaluate
import numpy as np
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
        
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()
print("Đã dọn dẹp bộ nhớ GPU!")

In [ ]:
import os
import torch
import gc
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
base_path = "/kaggle/input/vlsp-dataset"
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read().splitlines()

In [ ]:
print("Đang load dữ liệu...")
train_dataset = Dataset.from_dict({"en": read_text_file(os.path.join(base_path, "train.en.txt")), "vi": read_text_file(os.path.join(base_path, "train.vi.txt"))})
val_data_full = Dataset.from_dict({"en": read_text_file(os.path.join(base_path, "public_test.en.txt")), "vi": read_text_file(os.path.join(base_path, "public_test.vi.txt"))})
val_dataset = val_data_full.select(range(500)) 

raw_datasets = DatasetDict({'train': train_dataset, 'validation': val_dataset})

In [ ]:
model_checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang="eng_Latn", tgt_lang="vie_Latn")

In [ ]:
def preprocess_vi_en(examples):
    tokenizer.src_lang = "vie_Latn"
    tokenizer.tgt_lang = "eng_Latn"
    return tokenizer(examples["vi"], text_target=examples["en"], max_length=128, truncation=True)

print("Đang xử lý dữ liệu Việt -> Anh...")
tokenized_datasets = raw_datasets.map(preprocess_vi_en, batched=True)

In [ ]:
print("Đang load model...")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, 
    inference_mode=False, 
    r=16,               
    lora_alpha=32, 
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir="medical_nllb_final_vie_to_eng",
    eval_strategy="steps",
    eval_steps=1000,                
    save_strategy="steps",
    save_steps=1000,                
    learning_rate=3e-4,
    
    max_steps=4000,                 
    
    per_device_train_batch_size=4,   
    gradient_accumulation_steps=8,  
    gradient_checkpointing=False,    
    
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,             
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
print("Bắt đầu train ")
trainer.train()

trainer.model.save_pretrained("./final_adapter")
tokenizer.save_pretrained("./final_adapter")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

base_model_id = "facebook/nllb-200-distilled-600M"
adapter_path = "./final_adapter"  

tokenizer = AutoTokenizer.from_pretrained(base_model_id, src_lang="eng_Latn")

print("Đang load Base Model...")
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id)

print("Đang gắn Adapter Y tế...")
model = PeftModel.from_pretrained(base_model, adapter_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval() 
print(f"Load model thành công trên thiết bị: {device}")

In [ ]:
def medical_translate(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    target_lang_id = tokenizer.convert_tokens_to_ids("vie_Latn")

    with torch.no_grad():
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=target_lang_id, 
            max_length=128,
            num_beams=5,
            early_stopping=True
        )
    
    result = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return result

In [ ]:
test_sentences = [
    "I love you",
    "Please prescribe 500mg of Paracetamol to be taken twice daily after meals.",
    "Common symptoms of type 2 diabetes include increased thirst and frequent urination.",
    "The MRI scan revealed a small tumor in the left hemisphere of the brain.",
    "Wash your hands thoroughly to prevent the spread of bacterial infection."
]

print("-" * 50)
print("KẾT QUẢ DỊCH Y TẾ (ANH -> VIỆT)")
print("-" * 50)

for en_text in test_sentences:
    vi_text = medical_translate(en_text)
    print(f"🇬🇧 En: {en_text}")
    print(f"🇻🇳 Vi: {vi_text}")
    print("-" * 30)

In [ ]:
import torch
from tqdm import tqdm
import evaluate

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
import os

os.makedirs("./export_model", exist_ok=True)

!cp -r ./final_adapter/* ./export_model/

!zip -r Medical_Translation_BLEU40.zip ./export_model